**This was my first attempt at doing some of my EDA in python before exporting the data to  Tableau. Still very basic but I found it very useful. I finished off the analysis in Tableau which is displayed here.**

In [ ]:
%%HTML
<div class='tableauPlaceholder' id='viz1545174579162' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Al&#47;AlphabettoUSNameRelationship-autoSize&#47;Dashboard4&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='AlphabettoUSNameRelationship-autoSize&#47;Dashboard4' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Al&#47;AlphabettoUSNameRelationship-autoSize&#47;Dashboard4&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1545174579162');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='800px';vizElement.style.height='1227px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
import matplotlib.pyplot as plt

from collections import Counter
import collections
import math

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
df = pd.read_csv('../input/NationalNames.csv')
df = df.dropna(axis=1,how='all')
firstLetter = []
for letter in df['Name']:
    firstLetter.append(letter[0])

#firstName

In [ ]:
firstLetter = [x.upper() for x in firstLetter]

#Counter(firstLetter).keys() # equals to list(set(words))
#Counter(firstLetter).values() # counts the elements' frequency...
counts = collections.Counter(firstLetter) # counts the elements' frequency...
counts = dict(sorted(counts.items(), key=lambda x: x[1], reverse=True))
alphaDict = {'A':1, 'B':2, 'C':3, 'D':4, 'E':5, 'F':6, 'G':7, 'H':8, 'I':9, 'J':10, 'K':11, 'L':12, 'M':13, 'N':14, 'O':15
             , 'P':16, 'Q':17, 'R':18, 'S':19, 'T':20, 'U':21, 'V':22, 'W':23, 'X':24, 'Y':25, 'Z':26}
dfAlpha = pd.DataFrame(list(alphaDict.items()), columns=['letter', 'alphaPosition'])

dfCount = pd.DataFrame(list(counts.items()), columns=['letter', 'count'])  ###convert dict to DataFrame
dfCount['popularity'] = dfCount.index.values + 1
dfCount['actualLoc'] = dfCount['popularity'] / 26

dfMerge = dfCount.merge(dfAlpha, left_on='letter', right_on='letter')
dfMerge['alphaLoc'] = dfMerge['alphaPosition'] / 26

dfMerge['score'] = dfMerge['alphaLoc'] - dfMerge['actualLoc']
dfMerge['absScore'] = abs(dfMerge['alphaLoc'] - dfMerge['actualLoc'])
dfMerge['score%'] = dfMerge['alphaPosition'] / dfMerge['popularity']
dfMerge['alpha-pop'] = dfMerge['alphaPosition'] - dfMerge['popularity']

dfMergeSortAlpha = dfMerge.sort_values(by=['letter'])
dfMergeSortScore = dfMerge.sort_values(by=['absScore'])
dfMerge.sort_values(by=['alpha-pop'])

In [ ]:
print(dfMerge['alphaPosition'].corr(dfMerge['popularity']))
print(dfMerge['alphaPosition'].corr(dfMerge['score']))

In [ ]:
dfMerge.to_csv("../working/alpha.csv", index=None, mode='w')

### Correlations

Now that we have dealt with the categorical variables and the outliers, let's continue with the EDA. One way to try and understand the data is by looking for correlations between the features and the target. We can calculate the Pearson correlation coefficient between every variable and the target using the `.corr` dataframe method.

The correlation coefficient is not the greatest method to represent "relevance" of a feature, but it does give us an idea of possible relationships within the data. Some [general interpretations of the absolute value of the correlation coefficent](http://www.statstutor.ac.uk/resources/uploaded/pearsons.pdf) are:


* .00-.19 “very weak”
*  .20-.39 “weak”
*  .40-.59 “moderate”
*  .60-.79 “strong”
* .80-1.0 “very strong”


In [ ]:
dfMerge['popularity'].corr(dfMerge['alphaPosition'])  #moderate

In [ ]:
plt.title("Count where 1st Letter")
plt.bar(range(len(counts)), list(counts.values()), align='center')
plt.xticks(range(len(counts)), list(counts.keys()))
plt.show()

plt.title("Alpha Order")
plt.bar(range(len(dfMergeSortAlpha['absScore'])), list(dfMergeSortAlpha['absScore']), align='center')
plt.xticks(range(len(dfMergeSortAlpha['letter'])), list(dfMergeSortAlpha['letter']))
plt.show() #lower bar = usage close to alpha position

plt.title("Score Order")
plt.bar(range(len(dfMergeSortScore['absScore'])), list(dfMergeSortScore['absScore']), align='center')
plt.xticks(range(len(dfMergeSortScore['letter'])), list(dfMergeSortScore['letter']))
plt.show() #lower bar = usage close to alpha position